In [2]:
import torch, time, os
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
import pandas as pd
import matplotlib.pyplot as plt
from models import *
from Losses import *
from Metrics import evaluate, cross_bound_check
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Load data

In [3]:
class TimeSeriesDataLoader:
    def __init__(self, data, window_size, label_column, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, scaler=None):
        self.data         = data
        self.window_size  = window_size
        self.label_column = label_column
        self.train_ratio  = train_ratio
        self.val_ratio    = val_ratio
        self.test_ratio   = test_ratio
        self.scaler       = scaler

        self.train_X, self.train_y = None, None
        self.val_X, self.val_y = None, None
        self.test_X, self.test_y = None, None

        self._prepare_data()

    def _prepare_data(self):
        # 删除缺失值
        self.data = self.data.dropna()

        # 划分特征和目标变量
        X = self.data.drop(columns=[self.label_column]).values
        y = self.data[self.label_column].values

        # 处理缺失值（如果有）
        # 进行标准化（如果需要）
        if self.scaler is None:
            self.scaler_x = StandardScaler()
            self.scaler_y = StandardScaler()
            X = self.scaler_x.fit_transform(X)
            y = self.scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

        data = np.concatenate((X, y.reshape(-1, 1)), axis=1)
        # 生成窗口数据
        sequences_X, sequences_y = [], []
        for i in range(len(self.data) - self.window_size + 1):
            window_X = data[i:i + self.window_size]
            window_y = y[i + self.window_size - 1]  # 取窗口最后一个样本作为目标变量
            sequences_X.append(window_X)
            sequences_y.append(window_y)
        sequences_X, sequences_y = np.array(sequences_X), np.array(sequences_y)
        # print(f'x: {sequences_X.shape}, y: {sequences_y.shape}')
        # sequences_X = np.concatenate((sequences_X, sequences_y.reshape(-1, 1)), axis=1)
        sequences_X = sequences_X.reshape(len(sequences_X), self.window_size*sequences_X.shape[2])

        # 划分训练集、验证集、测试集
        train_size = int(len(sequences_X) * self.train_ratio)
        val_size = int(len(sequences_X) * self.val_ratio)
        test_size = len(sequences_X) - train_size - val_size

        self.train_X, self.train_y = sequences_X[:train_size], sequences_y[:train_size]
        self.val_X, self.val_y = sequences_X[train_size:train_size + val_size], sequences_y[train_size:train_size + val_size]
        self.test_X, self.test_y = sequences_X[train_size + val_size:], sequences_y[train_size + val_size:]

    def inverse_transform(self, data, is_label=True):
        # 将标准化后的数据还原
        if is_label:
            if data.ndim < 2:
                return self.scaler_y.inverse_transform(data.reshape(-1, 1)).flatten()
            else:
                for i in range(data.shape[1]):
                    data[:, i] = self.scaler_y.inverse_transform(data[:, i].reshape(-1, 1)).flatten()
                return data
        else:
            return self.scaler_x.inverse_transform(data)

    def get_train_data(self, to_tensor=False):
        if to_tensor:
            return torch.from_numpy(self.train_X).to(torch.float32), torch.from_numpy(self.train_y).to(torch.float32)
        else:
            return self.train_X, self.train_y

    def get_val_data(self, to_tensor=False):
        if to_tensor:
            return torch.from_numpy(self.val_X).to(torch.float32), torch.from_numpy(self.val_y).to(torch.float32)
        else:
            return self.val_X, self.val_y

    def get_test_data(self, to_tensor=False):
        if to_tensor:
            return torch.from_numpy(self.test_X).to(torch.float32), torch.from_numpy(self.test_y).to(torch.float32)
        else:
            return self.test_X, self.test_y


In [4]:
data_path = '../data/Kaggle Wind Power Forecasting Data/Turbine_Data.csv'
df        = pd.read_csv(data_path, parse_dates=["Unnamed: 0"])
df        = df[['ActivePower', 'WindDirection','WindSpeed']]
df.dropna(axis=0, inplace=True)
df['WindDirection_sin'] = np.sin(df['WindDirection'])
df['WindDirection_cos'] = np.cos(df['WindDirection'])
df.drop('WindDirection', axis=1, inplace=True)
df.describe()

,ActivePower,WindSpeed,WindDirection_sin,WindDirection_cos
count,72162.000000,72162.000000,72162.000000,72162.000000
mean,572.129599,5.696706,0.000225,0.000490
std,595.556861,2.619927,0.687171,0.726505
min,-38.524659,0.000000,-0.999998,-1.000000
25%,51.347222,3.651712,-0.666370,-0.724097
50%,351.279387,5.330200,-0.026521,-0.013277
75%,961.237629,7.239115,0.683262,0.739171
max,1779.032433,22.970893,1.000000,1.000000


In [5]:
df = df.iloc[:1000]
df.describe()

,ActivePower,WindSpeed,WindDirection_sin,WindDirection_cos
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,381.634134,4.959830,0.051187,-0.000138
std,393.655087,1.723514,0.600078,0.798927
min,-10.175428,1.306637,-0.999981,-0.999912
25%,53.539986,3.558236,-0.387809,-0.871058
50%,263.965496,4.889602,0.123603,0.044258
75%,610.756459,6.309334,0.514004,0.857788
max,1706.169049,9.078578,0.999998,1.000000


In [6]:
window_size = 2
label_column = 'ActivePower'

loader = TimeSeriesDataLoader(df, window_size, label_column)
X_train, Y_train = loader.get_train_data()
X_val  , Y_val   = loader.get_val_data()
X_test , Y_test  = loader.get_test_data()

print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)
print(X_test.shape, Y_test.shape)

(699, 8) (699,)
(149, 8) (149,)
(151, 8) (151,)


## NESCQR

In [7]:
from models import QuantileRegressionEstimator
from Losses import PinballLoss

class NESCQR:
    def __init__(self, data_loader, model_pool:list, label_pool:list, batch_size:int, M:int, alpha_set:list, 
                 l_rate:float, max_epochs:int, replace, symmetric, saveflag, save_dir, 
                 alpha_base=None, step=2, device='cuda', verbose=True):
        assert 0 < M <= len(model_pool), "M must be in range (0, len(model_pool)]"
        self.data_loader = data_loader
        self.model_pool  = model_pool
        self.label_pool  = label_pool  # 与model_pool里每个模型一一对应的模型名字
        self.batch_size  = batch_size
        self.M           = M           # 最终的集成模型的基学习器个数
        self.alpha_set   = alpha_set   # 置信水平集合
        self.l_rate      = l_rate      # 学习率
        self.max_epochs  = max_epochs
        self.device      = device
        self.saveflag   = saveflag
        self.save_dir    = save_dir
        self.alpha_base  = alpha_base if alpha_base else max(alpha_set)
        self.quantiles   = [self.alpha_base / 2, 1 - self.alpha_base / 2]
        self.loss_fn     = PinballLoss(self.quantiles, self.device)
        self.replace     = replace    # 是否有放回地前向选择
        self.step        = step       # DMCQR算法更新步长，int, 越小更新越快越准确
        self.symmetric   = symmetric  # 是否采用对称性conformity score
        # self.logger      = logger
        self.verbose     = verbose
        
    def init_training(self, saveflag=False):
        # 先训练好每个基学习器
        X_train, Y_train = self.data_loader.get_train_data(to_tensor=True)
        X_val  , Y_val   = self.data_loader.get_val_data(to_tensor=True)
        # X_train, Y_train = X_train.to(self.device), Y_train.to(self.device)
        # X_val  , Y_val   = X_val.to(self.device), Y_val.to(self.device)

        assert len(X_train) == len(Y_train)
        assert len(X_val)   == len(Y_val)
        
        print(f'X_train.shape: {X_train.shape}, Y_train.shape: {Y_train.shape}')
        print(f'X_val.shape: {X_val.shape}, Y_val.shape: {Y_val.shape}')
        # print(f'X_test.shape: {X_test.shape}, Y_train.shape: {Y_test.shape}')

        num_models = len(self.model_pool)
        model_pool_trained = []
        for i, model in enumerate(self.model_pool):
            print(f'Model {i+1}/{num_models} {self.label_pool[i]} starts training...')

            # 采用DMCQR得到最终的预测区间，则只需要最大的alpha，即两条分位数即可得到多条预测区间上下界。
            learner = QuantileRegressionEstimator(model, [max(self.alpha_set)], self.max_epochs,
                                                   self.batch_size,self.device, self.l_rate, self.verbose)
            learner.fit(X_train, Y_train, X_val, Y_val)
            model_pool_trained.append(learner)
            print(f'Model {i+1}/{num_models} {self.label_pool[i]} finished training.')
            
            if saveflag:
                torch.save(learner, f'{self.save_dir}/trained_models/{self.label_pool[i]}.pth')
                print(f'Model {i+1}/{num_models} saved.')

        return model_pool_trained

    def forward_selection(self, model_pool_trained, label_pool, replace=True):
        # 前向选择出最优集成模型组合
        X_val  , Y_val   = self.data_loader.get_val_data(to_tensor=True)
        X_val  , Y_val   = X_val.to(self.device), Y_val.to(self.device)
        # pool = dict(zip(label_pool, model_pool_trained))
        if replace:
            print('Forward selection with replacement.')
        else:
            print('Forward selection without replacement.')

        selected_model, selected_label = [], []
        while len(selected_model) < self.M:
            best_loss = np.inf
            
            for i in range(len(model_pool_trained)):
                models_ = selected_model + [model_pool_trained[i]]
                merged_output = torch.stack([torch.from_numpy(model.predict(X_val)) for model in models_])
                merged_output = torch.mean(merged_output, axis=0)
                loss = self.loss_fn(merged_output, Y_val)
                if loss.item() < best_loss:
                    best_model = model_pool_trained[i]
                    best_loss  = loss.item()
                    best_label = i

            selected_model.append(best_model)
            selected_label.append(label_pool[best_label])
            if not replace:  # 无放回
                model_pool_trained.pop(best_label)
                label_pool.pop(best_label)
                
        print(f'Model selected: {selected_label}')

        return selected_model, selected_label

    def conformal(self, res_val, Y_val, res_test, Y_test, step, symmetric=True):
        # DMCQR
        assert res_val.shape[0] == Y_val.shape[0]
        assert res_val.shape[1] == 2
        assert res_test.shape[1] == 2

        Y_val  , Y_test   = np.array(Y_val),   np.array(Y_test)
        res_val, res_test = np.array(res_val), np.array(res_test)
        Y_all     = np.concatenate((Y_val, Y_test), axis=0)
        res_all   = np.concatenate((res_val, res_test), axis=0)
        num_alpha = len(self.alpha_set)
        conf_PI   = np.zeros((len(res_test), num_alpha*2))
        val_size  = len(Y_val)
        test_size = len(Y_test)

        if symmetric:  
            # DMCQRS, use symmetric conformity score, 对称误差集合
            print('Use symmetric conformity score to calibrate quantiles.')
            E = list(np.max((res_val[:, 0] - Y_val, Y_val - res_val[:, -1]), axis=0))  # 误差集合，队列，先进先出
            Q = np.zeros(num_alpha)

            for t in range(val_size, val_size + test_size):
                for i, alpha in enumerate(self.alpha_set):
                    Q[i] = np.quantile(E, (1-alpha)*(1+1/val_size))
                    conf_PI[:, i] = res_test[:, 0] - Q[i]
                    conf_PI[:, -(i+1)] = res_test[:, -1] + Q[i]

                    if t % step == 0:
                        # print(f't = {t}, Q = {Q}')
                        for j in range(t - self.step, t - 1):
                            e = np.max((res_all[j, 0] - Y_all[j], Y_all[j] - res_all[j, -1]),axis=0)
                            E.pop(0)
                            E.append(e)   

            return conf_PI
        
        else:   
            # DMCQRS, use asymmetric conformity score, 非对称误差集合
            print('Use asymmetric conformity score to calibrate quantiles.')
            Q_low, Q_high = np.zeros(num_alpha), np.zeros(num_alpha)
            E_low = list(res_val[:, 0] - Y_val)    # 下界误差集合
            E_high = list(Y_val - res_val[:,-1])   # 上界误差集合
                
            for t in range(val_size, val_size + test_size):
                for i, alpha in enumerate(self.alpha_set):
                    Q_low[i] = np.quantile(E_low, (1 - alpha / 2))
                    Q_high[-(i+1)] = np.quantile(E_high, (1 - alpha / 2))
                    
                    conf_PI[:, i] = res_test[:, 0] - Q_low[i]
                    conf_PI[:, -(i+1)] = res_test[:, -1] + Q_high[-(i+1)]

                if t % step == 0:
                    # print(f't: {t}, Q_low: {Q_low}, Q_high: {Q_high}')
                    for j in range(t - step, t - 1):
                        e_low = res_all[j, 0] - Y_all[j]
                        e_high = Y_all[j] - res_all[j, -1]
                        E_low.pop(0)
                        E_low.append(e_low)
                        E_high.pop(0)
                        E_high.append(e_high)

            return conf_PI     

    def fit(self):
        self.model_pool_trained = self.init_training()
        self.model_pool_selected, self.selected_label = self.forward_selection(self.model_pool_trained, self.label_pool, self.replace)

    def predict(self, X_test=None, Y_test=None, inverse_normalization=True):
        # construct prediction intervals
        X_val  , Y_val   = self.data_loader.get_val_data(to_tensor=True)
        if not X_test:
            X_test,  Y_test  = self.data_loader.get_test_data(to_tensor=True)
        Y_val  , Y_test  = Y_val.detach().numpy(), Y_test.detach().numpy()
        X_val   = X_val.to(self.device)
        X_test  = X_test.to(self.device)

        # pred = self.model_pool_selected[0].predict(X_val)
        # print(f'pred.shape: {pred.shape}')
        res_val = torch.stack([torch.from_numpy(model.predict(X_val)) for model in self.model_pool_selected])
        res_val = torch.mean(res_val, axis=0)
        res_val = res_val.detach().numpy()
        res_test = torch.stack([torch.from_numpy(model.predict(X_test)) for model in self.model_pool_selected])
        res_test = torch.mean(res_test, axis=0)
        res_test = res_test.detach().numpy()
        # print(f'res_val.shape: {res_val.shape}, res_test.shape: {res_test.shape}')

        self.conf_PI = self.conformal(res_val, Y_val, res_test, Y_test, self.step, self.symmetric)
        if inverse_normalization:  # 逆标准化回原来的量纲
            self.conf_PI = self.data_loader.inverse_transform(self.conf_PI, is_label=True)

        cols = [str(round(alpha/2, 3)) for alpha in self.alpha_set] + [str(round(1-alpha/2, 3)) for alpha in reversed(self.alpha_set)]
        if self.saveflag:
            df = pd.DataFrame(self.conf_PI, columns=cols)
            df.to_csv(os.path.join(self.save_dir,'conf_PIs.csv'), index=False)
            
        return self.conf_PI

In [8]:
# 获取当前时间戳
current_time = time.strftime("%Y_%m_%d_%H_%M_%S")
# 构建文件夹路径
save_dir = os.path.join("result", current_time)
print(save_dir)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

n_ensemble = 3  #number of baseline models, which is M in paper.
M          = n_ensemble
max_epochs = 100
l_rate     = 1e-4
activation = 'tanh'  #nn.Tanh,              nn.ReLU, nn.Sigmoid
batch_size = 512
dropout    = 0.2
replace    = False
symmetric  = True
saveflag   = True
# save_dir   = './results/'
step       = 2
device     = 'cuda'
verbose    = True

alpha_set = np.array([0.05, 0.10, 0.15])
num_alpha = len(alpha_set)
alpha_base = max(alpha_set)
quantiles = [max(alpha_set)/2, 1 - max(alpha_set)/2]
# quantiles = np.zeros(2*num_alpha)
# for i in range(num_alpha):
#     quantiles[i] = alpha_set[i] / 2
#     quantiles[-(i+1)] = 1 - alpha_set[i] / 2

# loss_fn = PinballLoss(quantiles=quantiles, device=device)
input_dim = X_train.shape[1]
x_size = len(df.columns)
out_dim = len(quantiles)
kernel_size = 2
num_repeat = 1
hidden_units = [20 + i*4 for i in range(num_repeat)]
channel_sizes = [3 + i*2 for i in range(num_repeat)]

model_pool = [NET(input_dim, h, out_dim, activation) for h in hidden_units] + \
             [RNN(input_dim, h, out_dim, activation, device) for h in hidden_units] + \
             [LSTM(input_dim, h, out_dim, device) for h in hidden_units] + \
             [GRU(x_size, h, out_dim, device) for h in hidden_units] + \
             [TCN(x_size, out_dim, [c]*2, kernel_size, dropout) for c in channel_sizes]
             
# label_pool = ['BPNN']*num_repeat + ['RNN']*num_repeat + ['LSTM']*num_repeat + ['GRU']*num_repeat + ['TCN']*num_repeat
label_pool = [f'BPNN_{h}' for h in hidden_units] + \
             [f'RNN_{h}' for h in hidden_units] + \
             [f'LSTM_{h}' for h in hidden_units] + \
             [f'GRU_{h}' for h in hidden_units] + \
             [f'TCN_{c}' for c in channel_sizes]

nescqr = NESCQR(loader, model_pool, label_pool, batch_size, M, alpha_set, 
                 l_rate, max_epochs, replace, symmetric, saveflag, save_dir, alpha_base, step, 
                  device, verbose)

print(label_pool)

result\2023_10_10_11_17_09


f:\Postgraduate\papers\NESCQR\code\NESCQR\venv\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


['BPNN_20', 'RNN_20', 'LSTM_20', 'GRU_20', 'TCN_3']


In [9]:
# X_test , Y_test  = loader.get_test_data(to_tensor=True)
nescqr.fit()
conf_PI = nescqr.predict()
conf_PI.shape

X_train.shape: torch.Size([699, 8]), Y_train.shape: torch.Size([699])
X_val.shape: torch.Size([149, 8]), Y_val.shape: torch.Size([149])
Model 1/5 BPNN_20 starts training...
Epoch:100, train_loss: 0.4481, validation_loss: 0.5012, cost_time: 0.02s
Model 1/5 BPNN_20 finished training.
Model 2/5 RNN_20 starts training...
Epoch:100, train_loss: 0.3219, validation_loss: 0.3993, cost_time: 0.02s
Model 2/5 RNN_20 finished training.
Model 3/5 LSTM_20 starts training...
Epoch:100, train_loss: 0.3702, validation_loss: 0.4294, cost_time: 0.01s
Model 3/5 LSTM_20 finished training.
Model 4/5 GRU_20 starts training...
Epoch:100, train_loss: 0.4170, validation_loss: 0.4651, cost_time: 0.02s
Model 4/5 GRU_20 finished training.
Model 5/5 TCN_3 starts training...
Epoch:100, train_loss: 0.5138, validation_loss: 0.6536, cost_time: 0.03s
Model 5/5 TCN_3 finished training.
Forward selection without replacement.
Model selected: ['RNN_20', 'LSTM_20', 'GRU_20']
Use symmetric conformity score to calibrate quanti

(151, 6)

In [10]:
conf_PI[0]

array([-237.68874139, -123.15764317,  -79.04773945,  837.89391033,
        882.00381405,  996.53491227])

In [11]:
Y_test_original = loader.inverse_transform(Y_test)
res = evaluate(Y_test_original, conf_PI, alpha_set)
res

PINC: 95%, MAE: 250.0304, MSE: 91057.0204, RMSE: 301.7566, CRPS: 250.0304, SDE: 298.6288
PINC: 95%, PICP: 96.0265, MPIW: 1247.3746, PINAW: 0.9626, F: 0.0016, ACE: 1.0265, CWC: 0.9626,                   interval_score: 1473.1454
PINC: 90%, MAE: 250.0304, MSE: 91057.0204, RMSE: 301.7566, CRPS: 250.0304, SDE: 298.6288
PINC: 90%, PICP: 92.7152, MPIW: 1018.3124, PINAW: 0.7858, F: 0.0020, ACE: 2.7152, CWC: 0.7858,                   interval_score: 1255.1450
PINC: 85%, MAE: 250.0304, MSE: 91057.0204, RMSE: 301.7566, CRPS: 250.0304, SDE: 298.6288
PINC: 85%, PICP: 89.4040, MPIW: 930.0926, PINAW: 0.7177, F: 0.0022, ACE: 4.4040, CWC: 0.7177,                   interval_score: 1140.4828


,PINC,MAE,MSE,RMSE,CRPS,SDE,PICP,MPIW,F,PINAW,ACE,CWC,Interval score
0,95.0,250.030378,91057.020361,301.756558,250.030378,298.628832,96.026490,1247.374634,0.001603,0.962574,1.026490,0.962574,1473.145385
1,90.0,250.030378,91057.020361,301.756558,250.030378,298.628832,92.715232,1018.312437,0.001964,0.785811,2.715232,0.785811,1255.145046
2,85.0,250.030378,91057.020361,301.756558,250.030378,298.628832,89.403974,930.092630,0.002150,0.717734,4.403974,0.717734,1140.482809


In [12]:
res_cross = cross_bound_check(conf_PI)
res_cross

No cross-bound phenomenon.
MUCW = 0.0000, MLCW = 0.0000
Cross loss:  0.0


,MUCW,MLCW,Cross loss
0,0,0,0.0


## EnbPI

In [27]:
from algorithms import EnbPI

X_train, Y_train = loader.get_train_data(to_tensor=True)
X_val  , Y_val   = loader.get_val_data(to_tensor=True)
X_test , Y_test  = loader.get_test_data(to_tensor=True)
print(f'X_train.shape: {X_train.shape}, Y_train.shape: {Y_train.shape}')
print(f'X_val.shape: {X_val.shape}, Y_val.shape: {Y_val.shape}')
print(f'X_test.shape: {X_test.shape}, Y_train.shape: {Y_test.shape}')

X_train_enpi = torch.cat((X_train, X_val), axis=0)
Y_train_enpi = torch.cat((Y_train, Y_val), axis=0)
print(f'X_train_enpi.shape: {X_train_enpi.shape}, Y_train_enpi.shape: {Y_train_enpi.shape}')

# Define models
model_pool_enbpi = [NET(input_dim, h, 1, activation) for h in hidden_units] + \
             [RNN(input_dim, h, 1, activation, device) for h in hidden_units] + \
             [LSTM(input_dim, h, 1, device) for h in hidden_units] + \
             [GRU(x_size, h, 1, device) for h in hidden_units] + \
             [TCN(x_size, 1, [c]*2, kernel_size, dropout) for c in channel_sizes]

enbpi = EnbPI(model_pool_enbpi, alpha_set, l_rate, max_epochs, batch_size, device, verbose)
enbpi.fit(X_train_enpi, Y_train_enpi)

X_train.shape: torch.Size([699, 8]), Y_train.shape: torch.Size([699])
X_val.shape: torch.Size([149, 8]), Y_val.shape: torch.Size([149])
X_test.shape: torch.Size([151, 8]), Y_train.shape: torch.Size([151])
X_train_enpi.shape: torch.Size([848, 8]), Y_train_enpi.shape: torch.Size([848])
-- EnbPI training: 1 of 5 NNs --
x_s_b.shape = torch.Size([848, 8]), y_s_b = torch.Size([848, 1]), x_no_s_b.shape = torch.Size([316, 8]), y_no_s_b.shape = torch.Size([316, 1])


f:\Postgraduate\papers\NESCQR\code\NESCQR\venv\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Epoch:100, train_loss: 1.0757, validation_loss: 0.9045, cost_time: 0.02s
model: 1 finished training.
-- EnbPI training: 2 of 5 NNs --
x_s_b.shape = torch.Size([848, 8]), y_s_b = torch.Size([848, 1]), x_no_s_b.shape = torch.Size([313, 8]), y_no_s_b.shape = torch.Size([313, 1])
Epoch:100, train_loss: 0.9396, validation_loss: 0.9948, cost_time: 0.02s
model: 2 finished training.
-- EnbPI training: 3 of 5 NNs --
x_s_b.shape = torch.Size([848, 8]), y_s_b = torch.Size([848, 1]), x_no_s_b.shape = torch.Size([313, 8]), y_no_s_b.shape = torch.Size([313, 1])
Epoch:100, train_loss: 1.0038, validation_loss: 0.9345, cost_time: 0.02s
model: 3 finished training.
-- EnbPI training: 4 of 5 NNs --
x_s_b.shape = torch.Size([848, 8]), y_s_b = torch.Size([848, 1]), x_no_s_b.shape = torch.Size([318, 8]), y_no_s_b.shape = torch.Size([318, 1])
Epoch:100, train_loss: 0.8260, validation_loss: 0.7725, cost_time: 0.03s
model: 4 finished training.
-- EnbPI training: 5 of 5 NNs --
x_s_b.shape = torch.Size([848, 8]),

In [14]:
conf_PI_enbpi = enbpi.predict_interval(X_train_enpi, Y_train_enpi, X_test, Y_test, step)
conf_PI_enbpi = loader.inverse_transform(conf_PI_enbpi, is_label=True)
print(f'conf_PI_enbpi.shape: {conf_PI_enbpi.shape}')

res_enbpi = evaluate(Y_test_original, conf_PI_enbpi, alpha_set)
res_enbpi

conf_PI_enbpi.shape: (151, 6)
PINC: 95%, MAE: 230.6093, MSE: 75816.3135, RMSE: 275.3476, CRPS: 230.6093, SDE: 275.2886
PINC: 95%, PICP: 98.6755, MPIW: 1420.9276, PINAW: 1.0965, F: 0.0014, ACE: 3.6755, CWC: 1.0965,                   interval_score: 1451.1636
PINC: 90%, MAE: 230.6093, MSE: 75816.3135, RMSE: 275.3476, CRPS: 230.6093, SDE: 275.2886
PINC: 90%, PICP: 96.6887, MPIW: 1016.9813, PINAW: 0.7848, F: 0.0020, ACE: 6.6887, CWC: 0.7848,                   interval_score: 1118.9812
PINC: 85%, MAE: 230.6093, MSE: 75816.3135, RMSE: 275.3476, CRPS: 230.6093, SDE: 275.2886
PINC: 85%, PICP: 91.3907, MPIW: 805.3581, PINAW: 0.6215, F: 0.0025, ACE: 6.3907, CWC: 0.6215,                   interval_score: 947.2069


,PINC,MAE,MSE,RMSE,CRPS,SDE,PICP,MPIW,F,PINAW,ACE,CWC,Interval score
0,95.0,230.609263,75816.313483,275.347623,230.609263,275.288584,98.675497,1420.927611,0.001408,1.096501,3.675497,1.096501,1451.163562
1,90.0,230.609263,75816.313483,275.347623,230.609263,275.288584,96.688742,1016.981342,0.001967,0.784784,6.688742,0.784784,1118.981241
2,85.0,230.609263,75816.313483,275.347623,230.609263,275.288584,91.390728,805.358087,0.002483,0.621479,6.390728,0.621479,947.206893


In [15]:
cross_bound_check(conf_PI_enbpi)

No cross-bound phenomenon.
MUCW = 0.0000, MLCW = 0.0000
Cross loss:  0.0


,MUCW,MLCW,Cross loss
0,0,0,0.0


## EnCQR

In [17]:
# class EnCQR():
#     def __init__(self, model_pool, alpha_set, l_rate:float, max_epochs:int, batch_size:int, device='cuda', verbose=True):

#         self.n_ensemble = len(model_pool)  #基学习器数量
#         self.NNs = model_pool
#         self.l_rate = l_rate  #学习率
#         self.max_epochs = max_epochs
#         self.batch_size = batch_size  #越大更新越慢，int.
#         self.device = device
#         self.verbose = verbose  #是否输出中间过程
#         self.alpha_set = alpha_set
#         self.num_alpha = len(alpha_set)

#     def fit(self, X_train, Y_train):
    
#         T = X_train.shape[0]
#         tb = int(np.floor(T / self.n_ensemble))

#         S = np.arange(0, T)
#         for b in range(self.n_ensemble):
#             print('-- EnCQR training: ' + str(b+1) + ' of ' + str(self.n_ensemble) + ' NNs --')

#             s_b = range(tb*b, (tb*b+tb))
#             no_s_b = np.delete(S, s_b, 0)
#             x_s_b, y_s_b = X_train[s_b, :], Y_train[s_b].reshape(len(s_b), 1)

#             x_no_s_b = X_train[no_s_b, :]
#             y_no_s_b = Y_train[no_s_b].reshape(len(no_s_b), 1)

#             if self.verbose:
#                 print(f'x_s_b.shape = {x_s_b.shape}, y_s_b = {y_s_b.shape}, x_no_s_b.shape = {x_no_s_b.shape}, y_no_s_b.shape = {y_no_s_b.shape}')

#             learner = QuantileRegressionEstimator(self.NNs[b], self.alpha_set, self.max_epochs, self.batch_size,\
#                                                    self.device, self.l_rate, self.verbose)
#             learner.fit(x_s_b, y_s_b, x_no_s_b, y_no_s_b)
#             print('Model: %d finished training.' % (b+1))  
        
#         self.no_s_b = no_s_b

#     def predict(self, x):
#         '''
#         分位数预测。Quantile regression.

#         out:
#         res: point forecasting results of x, ndarray, [N, ].
#         '''
#         n_ensemble = len(self.NNs)
#         P = torch.zeros(n_ensemble, x.shape[0], self.num_alpha*2)

#         for b in range(n_ensemble):

#             model = self.NNs[b]
#             model.eval()
#             with torch.no_grad():
#                 x = x.to(self.device)
#                 pred = model(x)
            
#             P[b, :, :] = pred.to(torch.float32)
        
#         res = P.mean(axis=0)
#         res = res.numpy()
#         res = res.squeeze()

#         return res

#     def predict_interval(self, X_train, Y_train, X_test, Y_test, step=None):
#         '''
#         区间预测。Interval prediction. fit完直接就可以调用来构造预测区间。
        
#         out:
#         C: prediction intervals.
#         '''
        
#         Y_train, Y_test = np.array(Y_train), np.array(Y_test)

#         res_train = self.predict(X_train)
#         res_test = self.predict(X_test)
#         T1 = res_test.shape[0]
#         if step == None:
#             step = self.batch_size

#         # Initialize the asymmetric conformity scores.
#         C = np.zeros((T1, self.num_alpha*2))
#         E_low, E_high = np.zeros((len(self.no_s_b), self.num_alpha)), np.zeros((len(self.no_s_b), self.num_alpha))
#         Q_low, Q_high = np.zeros((self.num_alpha,)), np.zeros((self.num_alpha,))
#         for i in range(self.num_alpha):
#             E_low[:, i] = (res_train[self.no_s_b, i].reshape((len(self.no_s_b),1)) - Y_train[self.no_s_b].reshape((len(self.no_s_b),1))).squeeze()
#             E_high[:, -(i+1)] = (Y_train[self.no_s_b].reshape((len(self.no_s_b),1)) - res_train[self.no_s_b, -(i+1)].reshape((len(self.no_s_b),1))).squeeze()

#         # Comformalize the prediction intervals.
#         for t in range(T1):
#             for i, alpha in enumerate(self.alpha_set):
                
#                 Q_low[i] = np.quantile(E_low[:, i], 1 - alpha / 2)
#                 Q_high[-(i+1)] = np.quantile(E_high[:, -(i+1)], 1 - alpha / 2)

#                 C[t, i] = res_test[t, i] - Q_low[i]
#                 C[t, -(i+1)] = res_test[t, -(i+1)] + Q_high[-(i+1)]

#             # Update the lists of conformity scores
#             if t % step == 0 and step < T1:
#                 # print('t = %d, Q_low[0] = %f, Q_high[-1] = %f, E_low.shape = %s, E_high.shape = %s.' % 
#                 #   (t,Q_low[0],Q_high[-1],str(E_low.shape), str(E_high.shape)))
#                 for j in range(t - step, t-1):
#                     for i in range(self.num_alpha):
#                         e_low = res_test[j, i] - Y_test[j]
#                         e_high = Y_test[j] - res_test[j, -(i+1)]
#                         E_low_temp = np.delete(E_low[:,i], 0, 0)    #删除第一个元素
#                         E_low_temp = np.append(E_low_temp, e_low)   #添加新的元素
#                         E_low[:,i] = E_low_temp
#                         E_high_temp = np.delete(E_high[:,-(i+1)], 0, 0)
#                         E_high_temp = np.append(E_high_temp, e_high)
#                         E_high[:,-(i+1)] = E_high_temp   

#         return C

In [102]:
from utils import asym_nonconformity

class EnCQR:
    def __init__(self, model_pool, alpha_set, step, batch_size, l_rate, max_epochs, device, verbose):
        """
        Parameters
        ----------
        train_data : list of data to train an ensemble of models
        test_x : input test data
        test_y : output test data


        Returns
        -------
        PI : original PI produced by the ensemble model
        conf_PI : PI after the conformalization
        """

        self.model_pool = model_pool
        self.alpha_set  = alpha_set
        self.num_alpha  = len(alpha_set)
        self.step       = step
        self.batch_size = batch_size
        self.l_rate     = l_rate
        self.max_epochs = max_epochs
        self.device     = device
        self.verbose    = verbose

    def fit(self, train_data, val_x, val_y):
        """
        Train models.

        Args:
        train_data: list, [[x1, y1], [x2, y2], ...]
        val_x: input validation data
        val_y: output validation data
        """

        B = len(self.model_pool)
        index = np.arange(B)
        num_alpha = len(self.alpha_set)
    
        # dict containing LOO predictions
        dct_lo = {}
        dct_hi = {}
        for key in index:
            dct_lo['pred_%s' % key] = []
            dct_hi['pred_%s' % key] = []
        
        # training a model for each sub set Sb
        self.ensemble_models = []
        half = len(self.alpha_set)  # number of the alpha_set
        for b in range(B):
            print(f'-- EnCQR training: {b+1}/{B} NNs --')
            x, y = train_data[b][0], train_data[b][1]
            # print(f'b: {b}, x.shape: {x.shape}, y.shape: {y.shape}')
            learner = QuantileRegressionEstimator(self.model_pool[b], self.alpha_set, self.max_epochs, \
                                                  self.batch_size, self.device, self.l_rate, self.verbose)
            learner.fit(x, y, val_x, val_y)
            self.ensemble_models.append(learner)
            # print(f'b: learner.quantiles: {learner.quantiles}')
            
            # Leave-one-out predictions for each Sb, TERRIBLE
            # 在小样本上训练的模型去预测剩下的未见过的大样本，分位数表现非常糟糕
            indx_LOO = index[np.arange(len(index))!=b]
            print(f'b: {b}, indx_LOO: {indx_LOO}')
            for i in range(len(indx_LOO)):
                x_ = train_data[indx_LOO[i]][0]
                # print(f'b: {b}, i: {i}, indx_LOO[i]: {indx_LOO[i]}, x_.shape: {x_.shape}')
                pred = learner.predict(x_)
                # print(f'i: {i}, pred.mean: {pred.mean(axis=0)}')
                dct_lo['pred_%s' %indx_LOO[i]].append(pred[:, :half])
                dct_hi['pred_%s' %indx_LOO[i]].append(pred[:, half:])

        f_hat_b_agg_low  = np.zeros((train_data[index[0]][0].shape[0], half, B))
        f_hat_b_agg_high = np.zeros((train_data[index[0]][0].shape[0], half, B))
        for b in range(B):
            f_hat_b_agg_low[:,:,b] = np.mean(dct_lo['pred_%s' %b],axis=0) 
            f_hat_b_agg_high[:,:,b] = np.mean(dct_hi['pred_%s' %b],axis=0)  
            
        print(f'f_hat_b_agg_low.shape: {f_hat_b_agg_low.shape}, mean: {f_hat_b_agg_low.mean(axis=0)}')
        # residuals on the training data
        E_low, E_high = [], []
        for i in range(self.num_alpha):
            epsilon_low, epsilon_hi = [], []
            for b in range(B):
                e_low, e_high = asym_nonconformity(label=train_data[b][1].detach().numpy(), 
                                                        low=f_hat_b_agg_low[:,i,b], 
                                                        high=f_hat_b_agg_high[:,i,b])
                epsilon_low.append(e_low)
                epsilon_hi.append(e_high)
            E_low.append(epsilon_low)
            E_high.append(epsilon_hi)
        self.E_low = np.array(E_low)
        self.E_low = self.E_low.reshape(self.E_low.shape[1]*self.E_low.shape[2], self.num_alpha)
        self.E_high = np.array(E_high)
        self.E_high = self.E_high.reshape(self.E_high.shape[1]*self.E_high.shape[2], self.num_alpha)
        # print(f'E_low.shape: {self.E_low.shape}, E_high.shape: {self.E_high.shape}')
        # print(f'E_low.mean: {self.E_low.mean(axis=0)}, E_high.mean: {self.E_high.mean(axis=0)}')
        print('EnCQR training is done.')

    def predict(self, test_x, test_y, step=None):

        # PI
        res_test = np.zeros((len(self.ensemble_models), test_y.shape[0], len(self.alpha_set)*2))
        for i, model in enumerate(self.ensemble_models):
            pred = model.predict(test_x)
            print(f'pred.shape: {pred.shape}')
            res_test[i, :, :] = model.predict(test_x)

        res_test = np.mean(res_test, axis=0)
        # print(f'res_test.shape: {res_test.shape}')
        # print(f'res_test.mean: {res_test.mean(axis=0)}')
        
        # Conformal
        test_size = res_test.shape[0]
        if step == None:
            step = self.step

        # Initialize the asymmetric conformity scores.
        C = np.zeros((test_size, self.num_alpha*2))
        Q_low, Q_high = np.zeros((self.num_alpha,)), np.zeros((self.num_alpha,))

        # Comformalize the prediction intervals.
        for t in range(test_size):
            for i, alpha in enumerate(self.alpha_set):
                
                Q_low[i] = np.quantile(self.E_low[:, i], 1 - alpha / 2)
                Q_high[-(i+1)] = np.quantile(self.E_high[:, -(i+1)], 1 - alpha / 2)

                C[t, i] = res_test[t, i] - Q_low[i]
                C[t, -(i+1)] = res_test[t, -(i+1)] + Q_high[-(i+1)]

            # Update the lists of conformity scores
            if t % step == 0 and step < test_size:
                # print('t = %d, Q_low[0] = %f, Q_high[-1] = %f, E_low.shape = %s, E_high.shape = %s.' % 
                #   (t,Q_low[0],Q_high[-1],str(E_low.shape), str(E_high.shape)))
                for j in range(t - step, t-1):
                    for i in range(self.num_alpha):
                        e_low = res_test[j, i] - test_y[j]
                        e_high = test_y[j] - res_test[j, -(i+1)]
                        E_low_temp = np.delete(self.E_low[:,i], 0, 0)    #删除第一个元素
                        E_low_temp = np.append(E_low_temp, e_low)   #添加新的元素
                        self.E_low[:,i] = E_low_temp
                        E_high_temp = np.delete(self.E_high[:,-(i+1)], 0, 0)
                        E_high_temp = np.append(E_high_temp, e_high)
                        self.E_high[:,-(i+1)] = E_high_temp   

        return res_test, C

In [112]:
# Define models
# alpha_set_encqr = [0.1]
alpha_set_encqr = alpha_set
out_dim_encqr = len(alpha_set_encqr) * 2
model_pool_encqr = [NET(input_dim, h, out_dim_encqr, activation) for h in hidden_units] + \
             [RNN(input_dim, h, out_dim_encqr, activation, device) for h in hidden_units] + \
             [LSTM(input_dim, h, out_dim_encqr, device) for h in hidden_units] + \
             [GRU(x_size, h, out_dim_encqr, device) for h in hidden_units] + \
             [TCN(x_size, out_dim_encqr, [c]*2, kernel_size, dropout) for c in channel_sizes]

time_steps_in=24
time_steps_out=6

B = len(model_pool_encqr)
batch_len = int(np.floor(X_train.shape[0]/B))
# to_del = time_steps_in//time_steps_out # make sure there are no overlapping windows across batches
to_del = 0
print(f'X_train.shape = {X_train.shape}')
print(to_del, batch_len)

train_data = []
for b in range(len(model_pool_encqr)):
    print(f'b: {b}, batch_len: {batch_len}, b*batch_len:{b*batch_len}, (b+1)*batch_len-to_del: {(b+1)*batch_len-to_del}')
    train_data.append([X_train[b*batch_len:(b+1)*batch_len-to_del], Y_train[b*batch_len:(b+1)*batch_len-to_del]])
    
print(len(train_data))

max_epochs_encqr = 200
encqr = EnCQR(model_pool_encqr, alpha_set_encqr, step, batch_size, l_rate, max_epochs_encqr, device, verbose)
encqr.fit(train_data, X_val, Y_val)

f:\Postgraduate\papers\NESCQR\code\NESCQR\venv\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


X_train.shape = torch.Size([699, 8])
0 139
b: 0, batch_len: 139, b*batch_len:0, (b+1)*batch_len-to_del: 139
b: 1, batch_len: 139, b*batch_len:139, (b+1)*batch_len-to_del: 278
b: 2, batch_len: 139, b*batch_len:278, (b+1)*batch_len-to_del: 417
b: 3, batch_len: 139, b*batch_len:417, (b+1)*batch_len-to_del: 556
b: 4, batch_len: 139, b*batch_len:556, (b+1)*batch_len-to_del: 695
5
-- EnCQR training: 1/5 NNs --


Epoch:100, train_loss: 0.3441, validation_loss: 0.4145, cost_time: 0.01s
Epoch:200, train_loss: 0.3068, validation_loss: 0.3854, cost_time: 0.01s
b: 0, indx_LOO: [1 2 3 4]
b: 0, i: 0, indx_LOO[i]: 1, x_.shape: torch.Size([139, 8])
b: 0, i: 1, indx_LOO[i]: 2, x_.shape: torch.Size([139, 8])
b: 0, i: 2, indx_LOO[i]: 3, x_.shape: torch.Size([139, 8])
b: 0, i: 3, indx_LOO[i]: 4, x_.shape: torch.Size([139, 8])
-- EnCQR training: 2/5 NNs --
Epoch:100, train_loss: 0.3488, validation_loss: 0.3896, cost_time: 0.01s
Epoch:200, train_loss: 0.3123, validation_loss: 0.3594, cost_time: 0.01s
b: 1, indx_LOO: [0 2 3 4]
b: 1, i: 0, indx_LOO[i]: 0, x_.shape: torch.Size([139, 8])
b: 1, i: 1, indx_LOO[i]: 2, x_.shape: torch.Size([139, 8])
b: 1, i: 2, indx_LOO[i]: 3, x_.shape: torch.Size([139, 8])
b: 1, i: 3, indx_LOO[i]: 4, x_.shape: torch.Size([139, 8])
-- EnCQR training: 3/5 NNs --
Epoch:100, train_loss: 0.3793, validation_loss: 0.4089, cost_time: 0.01s
Epoch:200, train_loss: 0.3624, validation_loss: 0.3

In [113]:
PI_encqr, conf_PI_encqr = encqr.predict(X_test, Y_test)
print(f'conf_PI_encqr.shape: {conf_PI_encqr.shape}')

conf_PI_encqr = loader.inverse_transform(conf_PI_encqr, is_label=True)

res_encqr = evaluate(Y_test_original, conf_PI_encqr, alpha_set_encqr)
res_encqr

pred.shape: (151, 6)
pred.shape: (151, 6)
pred.shape: (151, 6)
pred.shape: (151, 6)
pred.shape: (151, 6)
conf_PI_encqr.shape: (151, 6)
PINC: 95%, MAE: 284.6974, MSE: 115419.2395, RMSE: 339.7341, CRPS: 284.6974, SDE: 292.4923
PINC: 95%, PICP: 99.3377, MPIW: 1329.8381, PINAW: 1.0262, F: 0.0015, ACE: 4.3377, CWC: 1.0262,                   interval_score: 1330.9188
PINC: 90%, MAE: 273.1011, MSE: 107079.1513, RMSE: 327.2295, CRPS: 273.1011, SDE: 291.7999
PINC: 90%, PICP: 96.0265, MPIW: 1120.5914, PINAW: 0.8647, F: 0.0018, ACE: 6.0265, CWC: 0.8647,                   interval_score: 1154.8223
PINC: 85%, MAE: 233.7237, MSE: 77171.6341, RMSE: 277.7978, CRPS: 233.7237, SDE: 274.5907
PINC: 85%, PICP: 96.6887, MPIW: 962.3339, PINAW: 0.7426, F: 0.0021, ACE: 11.6887, CWC: 0.7426,                   interval_score: 1021.3419


,PINC,MAE,MSE,RMSE,CRPS,SDE,PICP,MPIW,F,PINAW,ACE,CWC,Interval score
0,95.0,284.697375,115419.239541,339.734072,284.697375,292.492320,99.337748,1329.838079,0.001504,1.026210,4.337748,1.026210,1330.918785
1,90.0,273.101081,107079.151264,327.229509,273.101081,291.799875,96.026490,1120.591359,0.001785,0.864738,6.026490,0.864738,1154.822341
2,85.0,233.723728,77171.634077,277.797830,233.723728,274.590671,96.688742,962.333909,0.002078,0.742614,11.688742,0.742614,1021.341939


In [114]:
conf_PI_encqr.mean(axis=0)

array([ -68.87225295,   11.0235057 ,  -15.8519336 ,  946.48197527,
       1131.61486491, 1260.96582629])

In [121]:
cross_bound_check(conf_PI_encqr)

Cross-bound phenomenon exists.
l.sum() =  120.0
u.sum() =  0.0
MUCW = 0.0000, MLCW = 36.2124
Cross loss:  28.778033424833914


,MUCW,MLCW,Cross loss
0,0,36.212359,28.778033


In [117]:
X_train[b*batch_len:(b+1)*batch_len-to_del].shape

torch.Size([139, 8])

In [118]:
print(len(train_data[0]))
train_data[0][0].shape, train_data[0][1].shape

2


(torch.Size([139, 8]), torch.Size([139]))

In [119]:
B = len(model_pool_encqr)
index = np.arange(B)
print(f'index: {index}')
for b in range(B):
    print(f'index[b]: {index[b]}')
    indx_LOO = index[np.arange(len(index))!=b]
    print(f'indx_LOO: {indx_LOO}')

index: [0 1 2 3 4]
index[b]: 0
indx_LOO: [1 2 3 4]
index[b]: 1
indx_LOO: [0 2 3 4]
index[b]: 2
indx_LOO: [0 1 3 4]
index[b]: 3
indx_LOO: [0 1 2 4]
index[b]: 4
indx_LOO: [0 1 2 3]
